In [0]:
%pip install nltk

Python interpreter will be restarted.
Requirement already satisfied: nltk in /local_disk0/.ephemeral_nfs/envs/pythonEnv-10af4cb5-25e2-4b3b-8f61-94988f4c32c3/lib/python3.8/site-packages (3.6.3)
Requirement already satisfied: tqdm in /local_disk0/.ephemeral_nfs/envs/pythonEnv-10af4cb5-25e2-4b3b-8f61-94988f4c32c3/lib/python3.8/site-packages (from nltk) (4.62.3)
Requirement already satisfied: click in /local_disk0/.ephemeral_nfs/envs/pythonEnv-10af4cb5-25e2-4b3b-8f61-94988f4c32c3/lib/python3.8/site-packages (from nltk) (8.0.1)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.8/site-packages (from nltk) (0.17.0)
Requirement already satisfied: regex in /local_disk0/.ephemeral_nfs/envs/pythonEnv-10af4cb5-25e2-4b3b-8f61-94988f4c32c3/lib/python3.8/site-packages (from nltk) (2021.9.24)
WARNING: You are using pip version 20.2.4; however, version 21.2.4 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-10af4cb5-25e2-4b3b-8f61-94988f4c32c3/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
import re
import pandas as pd
import nltk
import numpy as np
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Package wordnet is already up-to-date!
Out[1]: True

In [0]:
def dataframe(check):
  df = spark.read.format("csv") \
.option("inferSchema","true") \
.option("header", "true") \
.option("sep",",") \
.load(check)
  return df

In [0]:
def remove_newlines_tabs(text):
    Formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')
    return Formatted_text
def remove_links(text):
    remove_https = re.sub(r'http\S+', '', text)
    remove_com = re.sub(r"\ [a-zA-Z0-9]*\.com", " ", remove_https)
    remove_png = re.sub(r"\ [a-zA-Z0-9]*\.png", " ", remove_com)
    return remove_png
def remove_whitespace(text):
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', text)
    text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    return text 
def upper_casing_text(text):
    text = text.upper()
    return text
def removing_special_characters(text):
    Formatted_Text = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', text)
    return Formatted_Text
def spelling_correction(text):
    spell = Speller(lang='en')
    Corrected_text = spell(text)
    return Corrected_text


In [0]:
def clean(text):
  text=remove_links(text)
  text=remove_newlines_tabs(text)
  text=remove_whitespace(text)
  #text=upper_casing_text(text)
  #text=removing_special_characters(text)
  #text=spelling_correction(text)
  return text

In [0]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatization(text):
    # Converting words to their root forms
    lemma = [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]
    return lemma

In [0]:
Spd=spark.sql("select id,TextBody from commbi_manual_surcharge_work.EMAIL_DATA")
df=Spd.toPandas()

tpdoc=spark.sql("""
SELECT  
TPDOC_NO
FROM commbi_model_prod.rt_shipment_tpdoc
GROUP BY 1 order by 1 DESC
""")
tpdoc=tpdoc.toPandas()
tpdoc2=set(tpdoc['TPDOC_NO'])


Spd=spark.sql("select * from commbi_manual_surcharge_work.EMAIL_DATA")
df=Spd.toPandas()

path ='/FileStore/tables/Surcharge.csv'
surcharge=(dataframe(path).toPandas()).values.tolist()

In [0]:
index_text=df.columns.get_loc('TextBody')
index_id=df.columns.get_loc('id')

In [0]:
 df['TextBody'] = df['TextBody'].apply(lambda x : clean(x))

In [0]:
patternShip=re.compile(r'\d{9}')
# patternId=re.compile(r'')
patternCont=re.compile(r'[A-Z]{4}[0-9]{7}')

patternTpDocType2=re.compile(r'[RLDHTK1GUNSJZAM]{1}[Y8N0OWAKPEBS1CLGZV]{1}\d{7}')

Endpattern=re.compile(r'>>')
AheadMoney = r"\s[AFN|ARS|AWG|AUD|AZN|BSD|BBD|BYN|BZD|BMD|BOB|BAM|BWP|BGN|BRL|BND|KHR|CAD|KYD|CLP|CNY|COP|CRC|HRK|CUP|CZK|DKK|DOP|XCD|EGP|SVC|EUR|FKP|FJD|GHS|GIP|GTQ|GGP|GYD|HNL|HKD|HUF|ISK|INR|IDR|IRR|IMP|ILS|JMD|JPY|JEP|KZT|KPW|KRW|KGS|LAK|LBP|LRD|MKD|MYR|MUR|MXN|MNT|MZN|NAD|NPR|ANG|NZD|NIO|NGN|NOK|OMR|PKR|PAB|PYG|PEN|PHP|PLN|QAR|RON|RUB|SHP|SAR|RSD|SCR|SGD|SBD|SOS|ZAR|LKR|SEK|CHF|SRD|SYP|TWD|THB|TTD|TRY|TVD|UAH|GBP|USD|UYU|UZS|VEF|VND|YER|ZWD|$|£|€|¥]\s*\d{1,}"
BehindMoney = r"\s\d{1,}\s*[AFN|ARS|AWG|AUD|AZN|BSD|BBD|BYN|BZD|BMD|BOB|BAM|BWP|BGN|BRL|BND|KHR|CAD|KYD|CLP|CNY|COP|CRC|HRK|CUP|CZK|DKK|DOP|XCD|EGP|SVC|EUR|FKP|FJD|GHS|GIP|GTQ|GGP|GYD|HNL|HKD|HUF|ISK|INR|IDR|IRR|IMP|ILS|JMD|JPY|JEP|KZT|KPW|KRW|KGS|LAK|LBP|LRD|MKD|MYR|MUR|MXN|MNT|MZN|NAD|NPR|ANG|NZD|NIO|NGN|NOK|OMR|PKR|PAB|PYG|PEN|PHP|PLN|QAR|RON|RUB|SHP|SAR|RSD|SCR|SGD|SBD|SOS|ZAR|LKR|SEK|CHF|SRD|SYP|TWD|THB|TTD|TRY|TVD|UAH|GBP|USD|UYU|UZS|VEF|VND|YER|ZWD|$|£|€|¥]"

In [0]:
%python
%pip install nlpaug

Python interpreter will be restarted.
Collecting nlpaug
 Downloading nlpaug-1.1.7-py3-none-any.whl (405 kB)
Installing collected packages: nlpaug
Successfully installed nlpaug-1.1.7
WARNING: You are using pip version 20.2.4; however, version 21.2.4 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-10af4cb5-25e2-4b3b-8f61-94988f4c32c3/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
import re

pattern = r"\s[EUR|USD|$|BRL]{3}\s*\d*"

test_string = 'abyss EUR 500 and USD 200000'

result = re.findall(pattern, test_string)

print(result)

[' EUR 500', ' USD 200000']

In [0]:
AllDict={}

for row in range(0,len(df)):
  
  templist=[]
  ContainerDict={}
  shipmentDict={}
  shipmentNm={}
  ContainerNm=[]
  TpDoc={}
  costs=[]
  
  shipmentNm=set(re.findall(patternShip,df.iat[row,index_text]))
  ContainerNm=list(set(re.findall(patternCont,df.iat[row,index_text])))
  tpdoc1=set(re.findall(patternTpDocType2,df.iat[row,index_text]))

  #   Set operations
  TpDoc=shipmentNm & tpdoc2
  TpDoc=tpdoc1 ^ TpDoc
  shipmentNm=shipmentNm-tpdoc2
  
  TpDoc=list(TpDoc)
  shipmentNm=list(shipmentNm)
  
  id=df.iat[row,index_id]
    
  if re.search(Endpattern,df.iat[row,index_text]) is not None:
    Index=re.search(Endpattern,df.iat[row,index_text]).start()
    ToSearch=df.iat[row,index_text][:Index]
    costs = re.findall(AheadMoney,ToSearch)
    if not costs:
      costs = re.findall(BehindMoney,ToSearch)
  
  if shipmentNm is None:
    shipmentNm[id]=''
  if ContainerNm is None:
    ContainerNm[id]=''
  if TpDoc is None:
    TpDoc[id]=''
  if Index is None:
    costs[id]=''
    
  else:
    templist.extend([shipmentNm,ContainerNm,TpDoc,costs])
    AllDict[id]=templist

In [0]:
print(AllDict)

In [0]:
cities_record = '[0-9]{9}'

In [0]:
df = pd.DataFrame(surcharge, columns=['surcharge'])

In [0]:
Spd=spark.sql("select * from commbi_manual_surcharge_work.EMAIL_DATA")
df=Spd.toPandas()

In [0]:
txt_list=df['TextBody'].tolist()
list_id = df['id'].tolist()

In [0]:
myDict = {}
myDict2 = {}
new_dict = {}
for strg in list_id:
  myDict[strg] = ['']
for strg in list_id:
  myDict2[strg] = ['']  

In [0]:
i=0
for surc in surcharge :
  i=0
  for strg in txt_list :
    if strg.find(surc[0])!= -1 :
      myDict[list_id[i]].append(surc[0])
    i=i+1

j=0
for i in txt_list:
  lst = re.findall(cities_record, i)
  if( len(lst) > 0 ):
    myDict2[list_id[j]].append(lst)
  j=j+1    
    

In [0]:
for key in myDict.keys():
  local_lst = myDict[key]
  if( len(local_lst) > 1 ):
    new_dict[key] = local_lst
    
for key in new_dict:
  if key in myDict2.keys():
    for it in myDict2[key]:
      new_dict[key].append(it)

In [0]:
AllDict.update(new_dict)

In [0]:
import json
json_object = json.dumps(AllDict, indent = 4) 
print(json_object)

{
 "02s2o00003i2UTGAA2": [
 [
 "212144821"
 ],
 [
 "MSKU9108898",
 "TRLU7204195",
 "CAIU8210535",
 "SUDU6578844",
 "MRKU4363820",
 "BMOU4098577",
 "TCKU7126542",
 "MRKU5946205",
 "TCNU9409211",
 "MRKU4132626",
 "MSKU0750438",
 "HASU4208154",
 "TCKU6134559",
 "MRKU5271400",
 "MRKU2909070",
 "SUDU5613663",
 "CAAU5641705",
 "CADU7020050",
 "CAIU7228571",
 "MSKU8979288",
 "TGHU8014332",
 "AMFU8782430",
 "APMU8043408",
 "HASU5005022",
 "MSKU1477009",
 "TLLU8071809",
 "TCNU8202723",
 "CAXU8143681",
 "UESU4620219",
 "PONU8232477",
 "FSCU8268771",
 "SEKU4707417"
 ],
 [
 "212576493",
 "268216966",
 "293216987",
 "212576317",
 "268216980",
 "268216964",
 "212577282",
 "212576836",
 "268217054",
 "212570298",
 "293216984",
 "212578487",
 "293217069"
 ],
 [
 " 7 D",
 " 42396 G",
 " 0 F",
 " 982 M",
 " 20457 H"
 ]
 ],
 "02s2o00003hbRnlAAE": [
 "",
 "Import Service",
 "",
 [
 "912541084",
 "977812925",
 "637672218",
 "934597979",
 "168651055",
 "168651055",
 "977812925",
 "637672218",
 "934607975",
 "977812925",
 "637672218",
 "934607975",
 "977812925",
 "637672218",
 "934617972",
 "977812925",
 "637672218",
 "934627968",
 "977812925",
 "637672218",
 "934627968",
 "977812925",
 "637672218",
 "934637963",
 "977812925",
 "637672218",
 "934637963",
 "977812925",
 "637672218",
 "934647956",
 "977812925",
 "637672218",
 "934657956",
 "977812925",
 "637672218",
 "934657956",
 "977812925",
 "637672218",
 "934667948",
 "977812925",
 "637672218",
 "934667948",
 "977812925",
 "637672218",
 "934677943",
 "977812925",
 "637672218",
 "934687936",
 "977812925",
 "637672218",
 "934687936",
 "977812925",
 "637672218",
 "934697932",
 "977812925",
 "637672218",
 "934707932",
 "977812925",
 "637672218",
 "934707932",
 "977812925",
 "637672218",
 "934717924",
 "977812925",
 "637672218",
 "934717924",
 "977812925",
 "637672218",
 "934727922",
 "977812925",
 "637672218",
 "934737918",
 "977812925",
 "637672218",
 "934737918",
 "977812925",
 "637672218",
 "934747912",
 "977812925",
 "637672218",
 "934747912",
 "977812925",
 "637672218",
 "934757909",
 "977812925",
 "637672218",
 "934767901",
 "977812925",
 "637672218",
 "934767901",
 "212851106",
 "977812925",
 "637672218",
 "934777900",
 "977812925",
 "637672218",
 "934777900",
 "977812925",
 "637672218",
 "934787893",
 "977812925",
 "637672218",
 "934797889",
 "977812925",
 "637672218",
 "934797889",
 "977812925",
 "637672218",
 "934807880",
 "977812925",
 "637672218",
 "934807880",
 "168651055",
 "977812925",
 "637672218",
 "934817877",
 "977812925",
 "637672218",
 "934827873",
 "977812925",
 "637672218",
 "934827873",
 "977812925",
 "637672218",
 "934837868",
 "977812925",
 "637672218",
 "934837868",
 "977812925",
 "637672218",
 "934847865",
 "977812925",
 "637672218",
 "934857859",
 "977812925",
 "637672218",
 "934857859",
 "977812925",
 "637672218",
 "934867854",
 "977812925",
 "637672218",
 "934867854",
 "977812925",
 "637672218",
 "934877850",
 "977812925",
 "637672218",
 "934887845",
 "977812925",
 "637672218",
 "934887845",
 "977812925",
 "637672218",
 "934897841",
 "977812925",
 "637672218",
 "934897841",
 "977812925",
 "637672218",
 "934907837",
 "977812925",
 "637672218",
 "934917833",
 "977812925",
 "637672218",
 "934917833",
 "977812925",
 "637672218",
 "934927828",
 "977812925",
 "637672218",
 "934927828",
 "977812925",
 "637672218",
 "934937825",
 "977812925",
 "637672218",
 "934947818",
 "977812925",
 "637672218",
 "934947818",
 "977812925",
 "637672218",
 "934957815"
 ]
 ],
 "02s2o00003SjSuAAAV": [
 "",
 "Basic Ocean Freight",
 "Terminal Handling Service - Destination",
 "Documentation Fee - Origin",
 "",
 [
 "315474731",
 "209325609",
 "209325609",
 "541977610",
 "113008190",
 "862152277",
 "862152277",
 "113008190",
 "113008190",
 "113002484",
 "113008190",
 "113008190",
 "113008190",
 "113008190",
 "113008190",
 "113008190",
 "113008190"
 ]
 ],
 "02s2o00003g9ZDZAA2": [
 "",
 "Cancellation Fee",
 "Export Service",
 "",
 [
 "544842426",
 "912071114",
 "912071114",
 "544854331",
 "544842426",
 

In [0]:
print(AllDict)

{'02s2o00003i2UTGAA2': [['212144821'], ['MSKU9108898', 'TRLU7204195', 'CAIU8210535', 'SUDU6578844', 'MRKU4363820', 'BMOU4098577', 'TCKU7126542', 'MRKU5946205', 'TCNU9409211', 'MRKU4132626', 'MSKU0750438', 'HASU4208154', 'TCKU6134559', 'MRKU5271400', 'MRKU2909070', 'SUDU5613663', 'CAAU5641705', 'CADU7020050', 'CAIU7228571', 'MSKU8979288', 'TGHU8014332', 'AMFU8782430', 'APMU8043408', 'HASU5005022', 'MSKU1477009', 'TLLU8071809', 'TCNU8202723', 'CAXU8143681', 'UESU4620219', 'PONU8232477', 'FSCU8268771', 'SEKU4707417'], ['212576493', '268216966', '293216987', '212576317', '268216980', '268216964', '212577282', '212576836', '268217054', '212570298', '293216984', '212578487', '293217069'], [' 7 D', ' 42396 G', ' 0 F', ' 982 M', ' 20457 H']], '02s2o00003hbRnlAAE': ['', 'Import Service', '', ['912541084', '977812925', '637672218', '934597979', '168651055', '168651055', '977812925', '637672218', '934607975', '977812925', '637672218', '934607975', '977812925', '637672218', '934617972', '977812925', '637672218', '934627968', '977812925', '637672218', '934627968', '977812925', '637672218', '934637963', '977812925', '637672218', '934637963', '977812925', '637672218', '934647956', '977812925', '637672218', '934657956', '977812925', '637672218', '934657956', '977812925', '637672218', '934667948', '977812925', '637672218', '934667948', '977812925', '637672218', '934677943', '977812925', '637672218', '934687936', '977812925', '637672218', '934687936', '977812925', '637672218', '934697932', '977812925', '637672218', '934707932', '977812925', '637672218', '934707932', '977812925', '637672218', '934717924', '977812925', '637672218', '934717924', '977812925', '637672218', '934727922', '977812925', '637672218', '934737918', '977812925', '637672218', '934737918', '977812925', '637672218', '934747912', '977812925', '637672218', '934747912', '977812925', '637672218', '934757909', '977812925', '637672218', '934767901', '977812925', '637672218', '934767901', '212851106', '977812925', '637672218', '934777900', '977812925', '637672218', '934777900', '977812925', '637672218', '934787893', '977812925', '637672218', '934797889', '977812925', '637672218', '934797889', '977812925', '637672218', '934807880', '977812925', '637672218', '934807880', '168651055', '977812925', '637672218', '934817877', '977812925', '637672218', '934827873', '977812925', '637672218', '934827873', '977812925', '637672218', '934837868', '977812925', '637672218', '934837868', '977812925', '637672218', '934847865', '977812925', '637672218', '934857859', '977812925', '637672218', '934857859', '977812925', '637672218', '934867854', '977812925', '637672218', '934867854', '977812925', '637672218', '934877850', '977812925', '637672218', '934887845', '977812925', '637672218', '934887845', '977812925', '637672218', '934897841', '977812925', '637672218', '934897841', '977812925', '637672218', '934907837', '977812925', '637672218', '934917833', '977812925', '637672218', '934917833', '977812925', '637672218', '934927828', '977812925', '637672218', '934927828', '977812925', '637672218', '934937825', '977812925', '637672218', '934947818', '977812925', '637672218', '934947818', '977812925', '637672218', '934957815']], '02s2o00003SjSuAAAV': ['', 'Basic Ocean Freight', 'Terminal Handling Service - Destination', 'Documentation Fee - Origin', '', ['315474731', '209325609', '209325609', '541977610', '113008190', '862152277', '862152277', '113008190', '113008190', '113002484', '113008190', '113008190', '113008190', '113008190', '113008190', '113008190', '113008190']], '02s2o00003g9ZDZAA2': ['', 'Cancellation Fee', 'Export Service', '', ['544842426', '912071114', '912071114', '544854331', '544842426', '912071114', '912071114', '544854331', '544842426', '912071114', '912071114', '544854331', '544840997', '912182872', '912182872', '544840997', '912182872', '912182872', '544854336', '912407538', '912407538', '544854336', '912407538', '912407538', '544836905', '912328587', '912328587', '544835493', '912328587', 